<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_10_3_streamlit_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 10: StreamLit**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 10 Material

Module 10: StreamLit

* Part 10.1: Running StreamLit in Google Colab [[Video]]() [[Notebook]](t81_559_class_10_1_streamlit.ipynb)
* Part 10.2: StreamLit Introduction [[Video]]() [[Notebook]](t81_559_class_10_2_streamlit_intro.ipynb)
* **Part 10.3: Understanding Streamlit State** [[Video]]() [[Notebook]](t81_559_class_10_3_streamlit_state.ipynb)
* Part 10.4: Creating a Chat Application [[Video]]() [[Notebook]](t81_559_class_10_4_chat.ipynb)
* Part 10.5: MultiModal Chat Application [[Video]]() [[Notebook]](t81_559_class_10_5_chat_multimodal.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain openai streamlit

# Part 10.3: Understanding Streamlit State

Streamlit is a powerful framework for building interactive web applications in Python. However, as you build more dynamic and complex apps, you may encounter the need to retain information between user interactions, such as keeping track of previous inputs, storing calculations, or remembering selections. This is where Streamlit's state management becomes essential.

In Streamlit, state refers to the mechanism for storing and managing data during a user's session. Without state management, each interaction (like clicking a button or changing an input) triggers the entire app to rerun, resetting any values you may want to preserve. State allows you to keep these values intact, enabling more advanced and responsive behaviors in your app.

### How Streamlit Handles State
Streamlit offers a simple way to handle state through st.session_state, a special object that persists values during a user’s session. This object works like a dictionary where you can store, update, and retrieve values across different parts of your app. By using session state, you can:

* Retain user input values when navigating through different components of your app.
* Store results of calculations or user choices to be used or modified in subsequent interactions.
* Build more complex apps that involve multiple steps, data processing, and interactivity.

## The Power of Streamlit State: A Loan Calculator Example
In this module, we will explore state management using a loan amortization calculator as a practical example. This app allows users to input various loan parameters, perform calculations, and store the results in session state for easy comparison. By leveraging state management, the app enables users to explore different loan scenarios without losing their previous calculations, offering a richer, more interactive experience.

Before diving into the specifics of the loan calculator example, we'll first cover the basics of how to use st.session_state effectively in Streamlit applications. This foundation will provide the knowledge needed to implement more advanced features and make your apps truly dynamic.

Let's get started by understanding how st.session_state works and how it can be utilized to maintain information across different parts of a Streamlit app.


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd

# Function to calculate loan amortization
def calculate_amortization(loan_amount, annual_rate, years):
    monthly_rate = annual_rate / 12 / 100
    num_payments = years * 12
    monthly_payment = loan_amount * (monthly_rate * (1 + monthly_rate) ** num_payments) / ((1 + monthly_rate) ** num_payments - 1)

    # Create amortization schedule
    amortization_data = []
    balance = loan_amount
    total_interest = 0
    for i in range(1, num_payments + 1):
        interest_payment = balance * monthly_rate
        principal_payment = monthly_payment - interest_payment
        balance -= principal_payment
        total_interest += interest_payment
        amortization_data.append([i, monthly_payment, principal_payment, interest_payment, max(balance, 0)])

    # Create DataFrame
    df = pd.DataFrame(amortization_data, columns=['Month', 'Payment', 'Principal', 'Interest', 'Balance'])
    return df, monthly_payment, total_interest

# Initialize session state to store calculations
if 'calculations' not in st.session_state:
    st.session_state['calculations'] = []

# Streamlit App
st.title('Loan Amortization Calculator')

# Input Fields
loan_amount = st.number_input('Loan Amount', value=300000.0, min_value=0.0, step=1000.0)
annual_rate = st.number_input('Annual Interest Rate (%)', value=7.5, min_value=0.0, step=0.1)
years = st.number_input('Loan Term (years)', value=30, min_value=1, step=1)

# Calculate amortization table
if st.button('Calculate Amortization Table'):
    amortization_df, monthly_payment, total_interest = calculate_amortization(loan_amount, annual_rate, years)
    st.write(f'Monthly Payment: ${monthly_payment:,.2f}')
    st.write(f'Total Interest Paid: ${total_interest:,.2f}')
    st.dataframe(amortization_df)

    # Save the current calculation to session state
    st.session_state['calculations'].append({
        'Loan Amount': loan_amount,
        'Annual Rate (%)': annual_rate,
        'Years': years,
        'Monthly Payment': monthly_payment,
        'Total Interest': total_interest
    })

# Display saved calculations
if st.session_state['calculations']:
    st.subheader('Saved Loan Calculations')
    comparison_df = pd.DataFrame(st.session_state['calculations'])
    st.dataframe(comparison_df)

    # Downloadable CSV of all saved calculations
    csv = comparison_df.to_csv(index=False)
    st.download_button(label="Download Comparison Table as CSV", data=csv, file_name='loan_comparisons.csv', mime='text/csv')

    # Clear all calculations
    if st.button('Clear All Calculations'):
        st.session_state['calculations'].clear()



The code provided demonstrates how to use Streamlit's session state to create an interactive loan amortization calculator that allows users to compare multiple loan scenarios. The app starts by defining a function to calculate the amortization schedule for a given loan amount, interest rate, and loan term. The main part of the app initializes the session state, creating a list to store loan calculations. This list is crucial for persisting user data throughout their interaction with the app.

When the user inputs loan parameters and clicks the "Calculate Amortization Table" button, the app computes the monthly payment and total interest using the specified values. These results are then stored in the session state list as a dictionary containing the loan amount, interest rate, loan term, monthly payment, and total interest. This allows the app to save each new calculation without losing previously entered data.

The app checks if there are any saved calculations in the session state. If present, it displays a comparison table showing all the stored loan scenarios, including the loan amount, interest rate, term, monthly payment, and total interest. Additionally, it provides a "Download Comparison Table as CSV" button, allowing users to download their comparison data for further analysis. A "Clear All Calculations" button is also included, which clears the session state, resetting the app to its initial state. This code highlights the power of st.session_state to build a more interactive and user-friendly app by preserving information across multiple user interactions.

Next, we obtain the password for our StreamLit server we are about to launch.


In [ ]:
!curl https://loca.lt/mytunnelpassword


We launch the StreamLit server and obtain its URL. You will need the above password when you access the URL it gives you.

In [ ]:
!streamlit run app.py server1 &>/content/logs.txt &
!npx --yes localtunnel --port 8501